In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import  MaxPool2D 
from tensorflow.keras.layers import  Dropout 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt

(train_images0, train_labels0), (test_images_raw, test_labels) = tensorflow.keras.datasets.fashion_mnist.load_data()

# Split the training set into a training and a validation set (20% is validation)
train_images_raw, val_images_raw, train_labels, val_labels = train_test_split(train_images0, train_labels0, test_size=0.20)

# Normalize the images.
train_images = (train_images_raw / 255) - 0.5
test_images = (test_images_raw / 255) - 0.5
val_images = (val_images_raw / 255) - 0.5

train_images_3d = np.expand_dims(train_images,3)
val_images_3d = np.expand_dims(val_images,3)
test_images_3d = np.expand_dims(test_images,3)

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
!pip install optuna

     |████████████████████████████████| 296kB 7.8MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 1.2MB 11.8MB/s 
     |████████████████████████████████| 112kB 34.3MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 143kB 27.2MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
  Created wheel for alembic: filename=alembic-1.6.2-py2.py3-none-any.whl size=164219 sha256=dcc9bc2ea1d1afb3d5f747b2482fe3ec3ff8b004cba0a57670df1880c74e89fd
  Stored in directory: /root/.cache/pip/wheels/a5/63/44/db29401e49ef5331c163b591f12a465c40af864bfa888dabd2
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=22dc8231f4fc4b80620ca77a2ef083e077a0b0dd226888efa5cd9c0365b572ab
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built alembic pyperclip


In [ ]:
import optuna

In [ ]:
def objective(trial):
  model = Sequential()

  model.add(Conv2D(filters=trial.suggest_categorical('conv_1_filter', [16,32,64,128,256]),
                    kernel_size=trial.suggest_categorical('conv_1_kernel', [3,5,7]),
                    input_shape=(28,28,1), data_format="channels_last", activation='relu'))
  model.add(MaxPool2D(2,2))
  model.add(Flatten())
  for i in range(trial.suggest_int("dense_nlayers", 1, 5)):
                 model.add(Dense(trial.suggest_int(f'dense_{i}_units', 16, 256, log=True),
                           trial.suggest_categorical('dense_activation', ['relu','tanh','elu'])))
                 model.add(Dropout(rate=trial.suggest_uniform(f"dropout_{i}_rate",0.1,0.5)))
                    
  model.add(Dense(10, activation='softmax'))
  

  model.compile(
    optimizer=Adam(trial.suggest_float('learning_rate', 1e-4, 0.1, log=True)),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  model.fit(train_images_3d, train_labels,
        epochs=10,
        validation_data=(val_images_3d, val_labels),
        batch_size=128,
        callbacks=[EarlyStopping('val_loss', patience=2)],
        verbose=False)
  val_acc = model.evaluate(val_images_3d, val_labels, verbose=0)

  return val_acc[1]

In [ ]:
study = optuna.create_study(study_name='fashion_mnist_cnn', direction='maximize')
study.optimize(objective, n_trials=20)

[I 2021-05-11 06:53:29,870] A new study created in memory with name: fashion_mnist_cnn
[I 2021-05-11 06:54:10,389] Trial 0 finished with value: 0.793833315372467 and parameters: {'conv_1_filter': 64, 'conv_1_kernel': 5, 'dense_nlayers': 5, 'dense_0_units': 71, 'dense_activation': 'tanh', 'dropout_0_rate': 0.11456293047583649, 'dense_1_units': 218, 'dropout_1_rate': 0.17991105757766915, 'dense_2_units': 29, 'dropout_2_rate': 0.19314748234853313, 'dense_3_units': 133, 'dropout_3_rate': 0.18677331862336227, 'dense_4_units': 136, 'dropout_4_rate': 0.2983386761386504, 'learning_rate': 0.007539959481430592}. Best is trial 0 with value: 0.793833315372467.
[I 2021-05-11 06:54:24,759] Trial 1 finished with value: 0.8845000267028809 and parameters: {'conv_1_filter': 64, 'conv_1_kernel': 5, 'dense_nlayers': 2, 'dense_0_units': 25, 'dense_activation': 'tanh', 'dropout_0_rate': 0.16442151623651174, 'dense_1_units': 31, 'dropout_1_rate': 0.3875295949676505, 'learning_rate': 0.00011480658661518222}. 

In [ ]:
study.best_params

{'conv_1_filter': 128,
 'conv_1_kernel': 3,
 'dense_0_units': 139,
 'dense_activation': 'tanh',
 'dense_nlayers': 1,
 'dropout_0_rate': 0.23902236025981177,
 'learning_rate': 0.001092537898600718}

In [ ]:
study.best_value

0.9169166684150696

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)

In [ ]:
study.optimize(objective, n_trials=20)

[I 2021-05-11 07:02:30,675] Trial 20 finished with value: 0.9041666388511658 and parameters: {'conv_1_filter': 64, 'conv_1_kernel': 7, 'dense_nlayers': 1, 'dense_0_units': 95, 'dense_activation': 'elu', 'dropout_0_rate': 0.2678866307355652, 'learning_rate': 0.002704094818049243}. Best is trial 13 with value: 0.9169166684150696.
[I 2021-05-11 07:02:49,602] Trial 21 finished with value: 0.9129999876022339 and parameters: {'conv_1_filter': 128, 'conv_1_kernel': 3, 'dense_nlayers': 1, 'dense_0_units': 130, 'dense_activation': 'elu', 'dropout_0_rate': 0.21703452543133672, 'learning_rate': 0.0008890945449834435}. Best is trial 13 with value: 0.9169166684150696.
[I 2021-05-11 07:03:11,883] Trial 22 finished with value: 0.9109166860580444 and parameters: {'conv_1_filter': 128, 'conv_1_kernel': 3, 'dense_nlayers': 1, 'dense_0_units': 180, 'dense_activation': 'elu', 'dropout_0_rate': 0.2516660679803939, 'learning_rate': 0.0009752838227103027}. Best is trial 13 with value: 0.9169166684150696.
[I 

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)

In [ ]:
from optuna.visualization import plot_slice

plot_slice(study, params=['learning_rate','dense_nlayers'])

In [ ]:
study.best_value

0.9175833463668823

In [ ]:
study.trials_dataframe().sort_values('value',ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_conv_1_filter,params_conv_1_kernel,params_dense_0_units,params_dense_1_units,params_dense_2_units,params_dense_3_units,params_dense_4_units,params_dense_activation,params_dense_nlayers,params_dropout_0_rate,params_dropout_1_rate,params_dropout_2_rate,params_dropout_3_rate,params_dropout_4_rate,params_learning_rate,state
35,35,0.917583,2021-05-11 07:06:59.459005,2021-05-11 07:07:26.086632,0 days 00:00:26.627627,128,3,130,NaN,NaN,NaN,NaN,elu,1,0.276725,NaN,NaN,NaN,NaN,0.000835,COMPLETE
31,31,0.917417,2021-05-11 07:05:47.920893,2021-05-11 07:06:13.569748,0 days 00:00:25.648855,128,3,121,NaN,NaN,NaN,NaN,elu,1,0.217993,NaN,NaN,NaN,NaN,0.001124,COMPLETE
13,13,0.916917,2021-05-11 06:57:44.773449,2021-05-11 06:58:08.981662,0 days 00:00:24.208213,128,3,139,NaN,NaN,NaN,NaN,tanh,1,0.239022,NaN,NaN,NaN,NaN,0.001093,COMPLETE
11,11,0.914500,2021-05-11 06:56:48.332566,2021-05-11 06:57:15.892668,0 days 00:00:27.560102,128,3,164,16.0,NaN,NaN,NaN,tanh,2,0.233739,0.26146,NaN,NaN,NaN,0.001344,COMPLETE
24,24,0.913750,2021-05-11 07:03:37.508569,2021-05-11 07:03:57.290539,0 days 00:00:19.781970,128,3,198,NaN,NaN,NaN,NaN,elu,1,0.194230,NaN,NaN,NaN,NaN,0.002310,COMPLETE


In [ ]:
objective(study.best_trial)

0.9175000190734863

In [ ]:
objective(study.best_trial)

0.9035000205039978